In [1]:
import config
import pandas as pd
import glob

from IPython.display import display
from IPython.core.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_seq_items', None)

In [2]:
import os
import os.path
curr_dir = os.getcwd()
data_dir = os.path.join(curr_dir,"..","data/")

In [3]:
!ls {data_dir}

1.csv  11.csv 13.csv 15.csv 3.csv  5.csv  7.csv  9.csv
10.csv 12.csv 14.csv 2.csv  4.csv  6.csv  8.csv  README


### Load/Shape Data

In [4]:
files = glob.glob(data_dir+'*.csv')
dfs = []
for n,f in enumerate(files):
    dfs.append(pd.read_csv(f,sep=',',names=config.cols))
    dfs[n]['participant'] = n+1
dfs = pd.concat(dfs).reset_index(drop=True)
print(dfs.shape)

(1926896, 6)


In [5]:
dfs.head(1)

,sequential_number,x_acceleration,y_acceleration,z_acceleration,label,participant
0,0.0,1502,2215,2153,1,1


In [6]:
dfs.groupby('label')['participant'].count().rename('counts').reset_index().T

,0,1,2,3,4,5,6,7
label,0,1,2,3,4,5,6,7
counts,3719,608667,47878,216737,357064,51498,47770,593563


In [7]:
X = dfs[['x_acceleration','y_acceleration','z_acceleration']].copy()
y = dfs['label'].copy()

### ML

In [8]:
from sklearn.neighbors import KNeighborsClassifier

In [9]:
knn = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
knn.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
           weights='uniform')

#### Score on all data (Overfitting can occur)

In [10]:
knn.score(X,y)

0.80655728176300123

#### Cross Validation

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
scores = cross_val_score(knn, X, y, cv=5,n_jobs=-1)
scores

array([ 0.25234962,  0.31122886,  0.26173647,  0.33153769,  0.26236263])

In [13]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.28 (+/- 0.06)
